# Pypowsybl action and operator strategy API demo 

In [ ]:
import pypowsybl as pp

In [ ]:
six_nodes = pp.network.load('data/reseau_6noeuds.xiidm')

![6 nodes network](data/reseau_6_noeuds_ss_HDVC.png)

In [ ]:
sa = pp.security.create_analysis()

In [ ]:
sa.add_single_element_contingency('S_SO_1', 'S_SO_1_contingency')

In [ ]:
sa.add_monitored_elements(branch_ids=['S_SO_2'])

## Topological remedial actions

In [ ]:
sa.add_switch_action(action_id='Switch_SS1_SS1_DJ_OMN_OPEN', switch_id='SS1_SS1_DJ_OMN', open=True)

In [ ]:
sa.add_switch_action(action_id='Switch_SOO1_SOO1_DJ_OMN_OPEN', switch_id='SOO1_SOO1_DJ_OMN', open=True)

In [ ]:
sa.add_operator_strategy('StrategyOpenSS1_SS1', 'S_SO_1_contingency', ['Switch_SS1_SS1_DJ_OMN_OPEN']) 

In [ ]:
sa.add_operator_strategy('StrategyOpenSOO1_SOO1', 'S_SO_1_contingency', ['Switch_SOO1_SOO1_DJ_OMN_OPEN']) 

In [ ]:
sa.add_operator_strategy('StrategyOpenBothSwitchs', 'S_SO_1_contingency', ['Switch_SS1_SS1_DJ_OMN_OPEN', 'Switch_SOO1_SOO1_DJ_OMN_OPEN']) 

## Redispatch remedial actions

In [ ]:
#sa.add_generator_active_power_action(action_id='Simple_Redispatch_Gen', generator_id='SO_G2', is_relative=False, active_power=380)
sa.add_generator_active_power_action(action_id='Simple_Redispatch_Gen', generator_id='SO_G2', is_relative=True, active_power=-580)

In [ ]:
sa.add_load_active_power_action(action_id='Simple_Redispatch_Load', load_id='SO_L', is_relative=False, active_power=500)

In [ ]:
sa.add_operator_strategy('StrategyRedispatch', 'S_SO_1_contingency', ['Simple_Redispatch_Gen', 'Simple_Redispatch_Load'])

## Run the security analysis 

In [ ]:
sa_result = sa.run_ac(six_nodes)

## Observe the flow on each step of the analysis

In [ ]:
print(sa_result.branch_results)